## Data Cleanup
### Introduction

This notebook outlines the data preparation steps necessary for sentiment analysis of reviews. Our focus is on transforming the dataset into a simplified format that facilitates subsequent analysis and modeling tasks. We'll retain key data for input (review `content`) and output (review `sentiment`), converting numerical scores into sentiment categories that are more meaningful for analysis.

### Setup and Preliminary Viewing

First, let's load the necessary libraries and preview the dataset to understand its current structure:

In [1]:
# Importing the pandas library for data manipulation
import pandas as pd

# Reading the dataset from a CSV file
df = pd.read_csv('../DATASETS/netflix_reviews.csv')

# Displaying the first few rows of the dataset to understand its initial structure
df.head()

,reviewId,userName,content,score,thumbsUpCount,reviewCreatedVersion,at,appVersion
0,cc1cfcd2-dc8a-4ead-88d1-7f2b2dbb2662,NR Bharadwaj,Plsssss stoppppp giving screen limit like when...,2,0,8.120.0 build 10 50712,2024-07-02 17:17:53,8.120.0 build 10 50712
1,7dfb1f90-f185-4e81-a97f-d38f0128e5a4,Maxwell Ntloko,Good,5,1,NaN,2024-06-26 15:38:06,NaN
2,3009acc4-8554-41cf-88de-cc5e2f6e45b2,Dilhani Mahanama,👍👍,5,0,NaN,2024-06-24 15:29:54,NaN
3,b3d27852-9a3b-4f74-9e16-15434d3ee324,Karen Gulli,Good,3,0,NaN,2024-06-22 15:41:54,NaN
4,8be10073-2368-4677-b828-9ff5d06ea0b7,Ronny Magadi,"App is useful to certain phone brand ,,,,it is...",1,0,8.105.0 build 15 50626,2024-06-22 05:16:03,8.105.0 build 15 50626


### Selecting Relevant Columns

Given our focus on sentiment analysis, we only need the text of the review and the associated score. However, we'll temporarily keep the reviewId to help identify and remove duplicate entries:

In [2]:
# Selecting only the necessary columns for sentiment analysis
df = df[['reviewId', 'content', 'score']]

# Displaying the updated DataFrame to verify the dropped columns
df.head()

,reviewId,content,score
0,cc1cfcd2-dc8a-4ead-88d1-7f2b2dbb2662,Plsssss stoppppp giving screen limit like when...,2
1,7dfb1f90-f185-4e81-a97f-d38f0128e5a4,Good,5
2,3009acc4-8554-41cf-88de-cc5e2f6e45b2,👍👍,5
3,b3d27852-9a3b-4f74-9e16-15434d3ee324,Good,3
4,8be10073-2368-4677-b828-9ff5d06ea0b7,"App is useful to certain phone brand ,,,,it is...",1


### Identifying and Removing Duplicates and Missing Values

It's essential to ensure our dataset is clean and free from duplicates and missing entries before proceeding:

In [3]:
# Function to display detailed information about missing and duplicated data
def show_details(dataset):
    missed_values = dataset.isnull().sum()
    missed_values_percent = 100 * (dataset.isnull().sum() / len(dataset))
    duplicated_values = dataset.duplicated().sum()
    duplicated_values_percent = 100 * (dataset.duplicated().sum() / len(dataset))
    info_frame = pd.DataFrame({'Missing_Values': missed_values, 
                               'Missing_Values %': missed_values_percent,
                               'Duplicated Values': duplicated_values,
                               'Duplicated Values %': duplicated_values_percent})
    return info_frame.T

# Displaying details of missing and duplicated data
show_details(df)

,reviewId,content,score
Missing_Values,0.000000,2.000000,0.000000
Missing_Values %,0.000000,0.001760,0.000000
Duplicated Values,316.000000,316.000000,316.000000
Duplicated Values %,0.278145,0.278145,0.278145


Given that these issues constitute only a small percentage of our data, we will remove them to ensure the quality of our dataset:

In [4]:
# Removing duplicate rows
df.drop_duplicates(inplace=True)

# Removing rows with missing values
df.dropna(inplace=True)

# Verifying that duplicates and missing values have been removed
show_details(df)

,reviewId,content,score
Missing_Values,0.0,0.0,0.0
Missing_Values %,0.0,0.0,0.0
Duplicated Values,0.0,0.0,0.0
Duplicated Values %,0.0,0.0,0.0


### Final Column Selection

Now that duplicates have been managed, we no longer need the reviewId:

In [5]:
# Removing the 'reviewId' column as it is no longer necessary
df = df[['content', 'score']]

### Transforming Scores to Sentiment Categories

Next, we'll convert numerical scores into categorical sentiments for easier interpretation during analysis:

In [6]:
# Defining a function to convert numerical scores to sentiment categories
def score_to_sentiment(score):
    if score <= 2:
        return 'negative'
    elif score == 3:
        return 'neutral'
    else:
        return 'positive'

# Applying the conversion function to the 'score' column
df['sentiment'] = df['score'].apply(score_to_sentiment)

# Displaying the updated DataFrame to verify the changes
df.head()

,content,score,sentiment
0,Plsssss stoppppp giving screen limit like when...,2,negative
1,Good,5,positive
2,👍👍,5,positive
3,Good,3,neutral
4,"App is useful to certain phone brand ,,,,it is...",1,negative


### Final Column Selection

With sentiments categorized, we no longer need the 'score' column:

In [7]:
# Removing the 'score' column as it is no longer necessary
df = df[['content', 'sentiment']]

# Displaying the updated DataFrame to ensure correct column selection
df.head()

,content,sentiment
0,Plsssss stoppppp giving screen limit like when...,negative
1,Good,positive
2,👍👍,positive
3,Good,neutral
4,"App is useful to certain phone brand ,,,,it is...",negative


### Saving the Cleaned Data

Finally, let's save the cleaned dataset for use later:

In [8]:
# Saving the cleaned dataset to a new CSV file
df.to_csv('../DATASETS/cleaned_data.csv', index=False)